In [1]:
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 74.7 MB/s eta 0:00:0031m57.8 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [2]:
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
import spacy

In [3]:
gpu = spacy.require_gpu()
print(gpu)

True


In [4]:
w2v_modelo_cbow = KeyedVectors.load_word2vec_format("modelo_cbow.txt")
w2v_modelo_sg = KeyedVectors.load_word2vec_format("modelo_skipgram.txt")

In [5]:
# download ==> https://cdn3.gnarususercontent.com.br/1638-word-embedding/treino.csv
artigo_treino = pd.read_csv("../introducao-word-embedding/treino.csv")

In [6]:
# download ==> https://cdn3.gnarususercontent.com.br/1638-word-embedding/teste.csv
artigo_teste = pd.read_csv("../introducao-word-embedding/teste.csv")

In [7]:
nlp = spacy.load("pt_core_news_sm", disable=["paser", "ner", "tagger", "textcat"])


def tokenizador(texto):

  doc = nlp(texto)
  tokens_validos = []
  for token in doc:
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      tokens_validos.append(token.text.lower())

  return tokens_validos


texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
tokens = tokenizador(texto)
print(tokens)

['rio', 'janeiro', 'cidade', 'maravilhosa']


In [8]:
def combinacao_de_vetores_por_soma(palavras, modelo):

  vetor_resultante = np.zeros((1, 300))

  for pn in palavras:
    try:
      vetor_resultante += modelo.get_vector(pn)

    except KeyError:
      pass

  return vetor_resultante


vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
print(vetor_texto.shape)
print(vetor_texto)

(1, 300)
[[-0.09780841  0.72173606  1.17411296 -1.873606    0.51987721  0.61661435
  -1.61286213  0.51550058  0.53943124 -0.99019702  0.16840574  0.39348035
  -0.89807041  0.37288859 -0.34885275 -1.80139849 -0.51982439 -1.18210216
   1.57076284  0.24765467 -0.50578925 -1.43960032 -0.84900111  0.27550486
  -0.25298117 -1.28743054  0.74339657  0.17569856  0.26850486  1.772517
  -0.48592576 -2.60330783 -0.9228661   2.11483273  1.89938051 -0.10421474
  -2.35170341 -1.98561734  1.29381786 -1.37437777  0.03397854  1.07148281
   1.76533729 -1.11733511 -1.92881085 -0.582232    2.85874599 -0.68514413
   1.4712524  -1.10186845 -1.78599739 -1.08320673  0.5267058  -1.0612969
  -0.96316433  0.51537016  1.28116336 -1.17951236  2.67347413  0.98597488
  -1.08900992 -2.37568307  1.14141631 -1.42069571  1.24463308  0.02878949
   0.40156133  1.39224318  0.59681375 -0.71936318  0.71016029  2.68591028
  -0.10795755 -0.07201315  0.95919481  0.49248065  0.94213992  0.35911627
  -2.4872793   0.57996921 -0.870

In [9]:
def matriz_vetores(textos, modelo):
  x = len(textos)
  y = 300
  matriz = np.zeros((x, y))

  for i in range(x):
    palavras = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

  return matriz

In [10]:
matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo_cbow)

In [11]:
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo_cbow)

In [12]:
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

(90000, 300)
(20513, 300)


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


def classificador(modelo, x_treino, y_treino, x_teste, y_teste):

  RL = LogisticRegression(max_iter=800)
  RL.fit(x_treino, y_treino)
  categorias = RL.predict(x_teste)
  resultados = classification_report(y_teste, categorias)
  print(resultados)

  return RL


RL_cbow = classificador(
    w2v_modelo_cbow,
    matriz_vetores_treino_cbow,
    artigo_treino.category,
    matriz_vetores_teste_cbow,
    artigo_teste.category,
)

              precision    recall  f1-score   support

     colunas       0.80      0.71      0.75      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.93      0.86      0.89      4663
   ilustrada       0.13      0.85      0.23       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.74      0.83      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.81      0.70     20513
weighted avg       0.82      0.79      0.80     20513



In [14]:
matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)

RL_sg = classificador(
    w2v_modelo_sg,
    matriz_vetores_treino_sg,
    artigo_treino.category,
    matriz_vetores_teste_sg,
    artigo_teste.category,
)

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.65      0.81      0.72      1698
     esporte       0.93      0.87      0.90      4663
   ilustrada       0.14      0.87      0.24       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.76      0.84      0.80      2051

    accuracy                           0.79     20513
   macro avg       0.69      0.82      0.70     20513
weighted avg       0.82      0.79      0.80     20513



In [15]:
import pickle

with open("rl_cbow.pkl", "wb") as f:
    pickle.dump(RL_cbow, f)

In [16]:
with open("rl_sg.pkl", "wb") as f:
    pickle.dump(RL_sg, f)